In [58]:
"""
Notebook for exploring, testing, etc
To be deleted before deadline

I'm committing this file just in case anything here might be useful
"""
import numpy as np
import pandas as pd
import sys
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
gdata = pd.read_json('data/genres.json.gz', orient='record', lines=True)
odata = pd.read_json('data/omdb-data.json.gz', orient='record', lines=True)
rdata = pd.read_json('data/rotten-tomatoes.json.gz', orient='record', lines=True)
wdata = pd.read_json('data/wikidata-movies.json.gz', orient='record', lines=True)

gd = gdata
od = odata
rd = rdata
wd = wdata

In [3]:
gdata.head()

,genre_label,wikidata_id
0,novella,Q43334491
1,cityscape,Q42304051
2,drama with music,Q39894018
3,lunar art,Q47909472
4,Tsukuri monogatari,Q42922249


In [4]:
odata.head()

,imdb_id,omdb_awards,omdb_genres,omdb_plot
0,tt0060814,Nominated for 2 Oscars. Another 2 nominations.,"[Drama, History, War]","In this sprawling, star-laden film, we see the..."
1,tt2379713,Won 1 Oscar. Another 7 wins & 32 nominations.,"[Action, Adventure, Thriller]",A cryptic message from the past sends James Bo...
2,tt0230575,N/A,"[Comedy, Horror]","The makers of this parody of ""Night of the Liv..."
3,tt0065988,Nominated for 1 Oscar. Another 5 wins & 8 nomi...,"[Adventure, Comedy, Drama]",Jack Crabb is 121 years old as the film begins...
4,tt1995390,2 nominations.,"[Crime, Drama, Thriller]","When Perry and his girlfriend, Gail, cross pat..."


In [5]:
rdata.head()

,audience_average,audience_percent,audience_ratings,critic_average,critic_percent,imdb_id,rotten_tomatoes_id
0,3.9,86.0,33125227.0,7.1,79.0,tt0325980,m/pirates_of_the_caribbean_the_curse_of_the_bl...
1,4.1,80.0,2139.0,7.8,88.0,tt4882376,m/first_they_killed_my_father_a_daughter_of_ca...
2,3.6,71.0,11482.0,7.6,86.0,tt0099763,m/henry_portrait_of_a_serial_killer
3,3.6,66.0,187204.0,6.6,66.0,tt1371111,m/cloud_atlas_2012
4,3.9,84.0,26836.0,7.4,80.0,tt1189073,m/the_skin_i_live_in


In [6]:
wdata[wdata['label'] == 'Deadpool']
wdata.head()

,based_on,cast_member,country_of_origin,director,enwiki_title,filming_location,genre,imdb_id,label,made_profit,main_subject,metacritic_id,original_language,publication_date,rotten_tomatoes_id,series,wikidata_id
0,NaN,"[Q228931, Q235384]",Q145,[Q43079418],If I Were You (2012 Canadian film),NaN,[Q859369],tt1587309,If I Were You,NaN,NaN,NaN,Q1860,2012-01-01,m/if_i_were_you_2012,NaN,Q43079072
1,NaN,NaN,Q30,NaN,Krampus: The Devil Returns,NaN,[Q28026639],tt3527772,Krampus: The Devil Returns,NaN,NaN,NaN,Q1860,2016-10-04,m/krampus_the_devil_returns,NaN,Q43302269
2,NaN,"[Q106303, Q467957, Q3345693, Q3340838, Q316419...",Q142,[Q724208],Le Brio,[Q50742],[Q157443],tt6462462,NaN,NaN,NaN,NaN,Q150,2017-01-01,m/le_brio,NaN,Q43400054
3,NaN,NaN,Q30,[Q4718041],A Christmas Prince,NaN,"[Q860626, Q28026639]",tt7608418,A Christmas Prince,NaN,NaN,NaN,Q1860,2017-11-17,m/a_christmas_prince,NaN,Q43401863
4,Q7162696,NaN,Q252,[Q11036795],Pengabdi Setan (2017 film),NaN,[Q200092],tt7076834,Pengabdi Setan,NaN,NaN,NaN,Q9240,2017-01-01,m/satans_slaves,NaN,Q41850450


In [8]:
wd2 = wd.join(od.set_index('imdb_id'), on='imdb_id')
rd = rd.drop(['rotten_tomatoes_id'], axis=1)
wd2 = wd2.join(rd.set_index('imdb_id'), on='imdb_id')
wd2.head()

,based_on,cast_member,country_of_origin,director,enwiki_title,filming_location,genre,imdb_id,label,made_profit,...,series,wikidata_id,omdb_awards,omdb_genres,omdb_plot,audience_average,audience_percent,audience_ratings,critic_average,critic_percent
0,NaN,"[Q228931, Q235384]",Q145,[Q43079418],If I Were You (2012 Canadian film),NaN,[Q859369],tt1587309,If I Were You,NaN,...,NaN,Q43079072,NaN,NaN,NaN,3.9,72.0,244.0,3.7,9.0
1,NaN,NaN,Q30,NaN,Krampus: The Devil Returns,NaN,[Q28026639],tt3527772,Krampus: The Devil Returns,NaN,...,NaN,Q43302269,NaN,NaN,NaN,1.8,24.0,31.0,NaN,NaN
2,NaN,"[Q106303, Q467957, Q3345693, Q3340838, Q316419...",Q142,[Q724208],Le Brio,[Q50742],[Q157443],tt6462462,NaN,NaN,...,NaN,Q43400054,NaN,NaN,NaN,3.4,67.0,13.0,6.7,80.0
3,NaN,NaN,Q30,[Q4718041],A Christmas Prince,NaN,"[Q860626, Q28026639]",tt7608418,A Christmas Prince,NaN,...,NaN,Q43401863,NaN,NaN,NaN,3.2,48.0,198.0,6.5,83.0
4,Q7162696,NaN,Q252,[Q11036795],Pengabdi Setan (2017 film),NaN,[Q200092],tt7076834,Pengabdi Setan,NaN,...,NaN,Q41850450,NaN,NaN,NaN,4.0,77.0,39.0,NaN,NaN


In [54]:
# drop empty plots
wd2['is_empty_plot'] = wd2['omdb_plot'].map(lambda plot: type(plot) == float)
wd2
wd2 = wd2[wd2['is_empty_plot'] == False]

#drop NaN
wd2 = wd2[np.isnan(wd2['audience_average']) == False]
wd2 = wd2[np.isnan(wd2['critic_average']) == False]
#X = wd2['omdb_plot'].values
y = np.around(wd2['critic_average'])
y = y.map(lambda num: str(num)).values
#X_train, X_test, y_train, y_test = train_test_split(X, y)

In [39]:
X_counts = CountVectorizer().fit_transform(wd2['omdb_plot'].values)
X_tf = TfidfTransformer(use_idf=True).fit_transform(X_counts)
X_tf.shape
#model = GaussianNB()
#model.fit(X_vec.toarray(), y_train)
#print(len(vectorizer.get_feature_names()))

#print(X_vec.toarray())

(6597, 33677)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X_tf, y)

In [57]:
model = MultinomialNB().fit(X_train, y_train)

array(['6.0', '7.0', '4.0', ..., '8.0', '5.0', '8.0'], dtype=object)